In [1]:
print('hello')

hello


In [1]:
import subprocess
import time
import re

import pyautogui
import pytesseract
import cv2
import numpy as np
from PIL import Image

# A) Maximize the Telegram Web window
subprocess.run([
    "wmctrl", "-r", "Telegram Web",
    "-b", "add,maximized_vert,maximized_horz"
], check=True)
time.sleep(1)  # wait for the resize


def get_window_geometry(win_name):
    """Return (x, y, width, height) for the first window matching win_name."""
    # 1. find window IDs
    out = subprocess.check_output(
        ["xdotool", "search", "--name", win_name]
    ).decode().splitlines()
    if not out:
        raise RuntimeError(f"No window found with name {win_name!r}")
    win_id = out[0]

    # 2. query its geometry
    geom = subprocess.check_output(
        ["xdotool", "getwindowgeometry", "--shell", win_id]
    ).decode().splitlines()
    d = {}
    for line in geom:
        if "=" in line:
            k, v = line.split("=", 1)
            d[k] = int(v)
    return d["X"], d["Y"], d["WIDTH"], d["HEIGHT"]


# B) Grab the window geometry via xdotool
win_x, win_y, win_w, win_h = get_window_geometry("Telegram Web")

# C) Define your message‐pane region (tweak offsets as needed)
LEFT   = win_x + 330
TOP    = win_y +  90
WIDTH  = win_w - 350
HEIGHT = win_h - 180

def capture_messages():
    im = pyautogui.screenshot(region=(LEFT, TOP, WIDTH, HEIGHT))
    gray = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2GRAY)
    return pytesseract.image_to_string(gray)

def scroll_up(pixels=500):
    pyautogui.moveTo(LEFT + WIDTH//2, TOP + HEIGHT//2, duration=0.1)
    pyautogui.scroll(pixels)

def scroll_down(pixels=500):
    pyautogui.moveTo(LEFT + WIDTH//2, TOP + HEIGHT//2, duration=0.1)
    pyautogui.scroll(-pixels)

def find_messages_by_date(date_str, direction="up", max_steps=50, pause=0.5):
    for step in range(max_steps):
        text = capture_messages()
        if re.search(re.escape(date_str), text):
            print(f"✅ Found “{date_str}” on step {step}")
            return text
        (scroll_up() if direction=="up" else scroll_down())
        time.sleep(pause)
    print(f"❌ Didn’t find “{date_str}” after {max_steps} steps.")
    return None


if __name__ == "__main__":
    result = find_messages_by_date("May 13", direction="up", max_steps=100)
    if result:
        print("--- OCR’d snippet ---")
        print("\n".join(result.splitlines()[:10]))


❌ Didn’t find “May 13” after 100 steps.


In [ ]:
import subprocess
import time
import re

import pyautogui
import pytesseract
import cv2
import numpy as np
from PIL import Image

# A) Maximize the Telegram Web window
subprocess.run([
    "wmctrl", "-r", "Telegram Web",
    "-b", "add,maximized_vert,maximized_horz"
], check=True)
time.sleep(1)  # wait for the resize


def get_window_geometry(win_name):
    """Return (x, y, width, height) for the first window matching win_name."""
    # 1. find window IDs
    out = subprocess.check_output(
        ["xdotool", "search", "--name", win_name]
    ).decode().splitlines()
    if not out:
        raise RuntimeError(f"No window found with name {win_name!r}")
    win_id = out[0]

    # 2. query its geometry
    geom = subprocess.check_output(
        ["xdotool", "getwindowgeometry", "--shell", win_id]
    ).decode().splitlines()
    d = {}
    for line in geom:
        if "=" in line:
            k, v = line.split("=", 1)
            d[k] = int(v)
    return d["X"], d["Y"], d["WIDTH"], d["HEIGHT"]


# B) Grab the window geometry via xdotool
win_x, win_y, win_w, win_h = get_window_geometry("Telegram Web")

# C) Define your message‐pane region (tweak offsets as needed)
LEFT   = win_x + 330
TOP    = win_y +  90
WIDTH  = win_w - 350
HEIGHT = win_h - 180

def capture_messages():
    im = pyautogui.screenshot(region=(LEFT, TOP, WIDTH, HEIGHT))
    gray = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2GRAY)
    return pytesseract.image_to_string(gray)

def scroll_up(pixels=500):
    pyautogui.moveTo(LEFT + WIDTH//2, TOP + HEIGHT//2, duration=0.1)
    pyautogui.scroll(pixels)

def scroll_down(pixels=500):
    pyautogui.moveTo(LEFT + WIDTH//2, TOP + HEIGHT//2, duration=0.1)
    pyautogui.scroll(-pixels)

def find_messages_by_date(date_str, direction="up", max_steps=50, pause=0.5):
    for step in range(max_steps):
        text = capture_messages()
        if re.search(re.escape(date_str), text):
            print(f"✅ Found “{date_str}” on step {step}")
            return text
        (scroll_up() if direction=="up" else scroll_down())
        time.sleep(pause)
    print(f"❌ Didn’t find “{date_str}” after {max_steps} steps.")
    return None


if __name__ == "__main__":
    result = find_messages_by_date("May 13", direction="up", max_steps=100)
    if result:
        print("--- OCR’d snippet ---")
        print("\n".join(result.splitlines()[:10]))


❌ Didn’t find “May 13” after 100 steps.


In [7]:
#!/usr/bin/env python3
"""
Simplified Telegram Channel Message Extractor
Goes directly to OptionRecom Premium Plus channel and extracts messages.
"""

import subprocess
import time
import re
import cv2
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import webbrowser
import warnings
import os

# Suppress warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
import pyautogui
import pytesseract

# Optional selenium imports
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    from webdriver_manager.chrome import ChromeDriverManager
    SELENIUM_AVAILABLE = True
except ImportError:
    SELENIUM_AVAILABLE = False

# Configuration
DAYS_TO_EXTRACT = 2
CHANNEL_URL = "https://web.telegram.org/k/#-1915245777"

class TelegramExtractor:
    def __init__(self, days=DAYS_TO_EXTRACT):
        self.days = days
        self.driver = None
        self.messages_data = []
        self.target_dates = self._get_target_dates()
        
    def _get_target_dates(self):
        """Get list of dates to search for"""
        dates = []
        for i in range(self.days):
            date = datetime.now() - timedelta(days=i)
            dates.append(date.strftime("%b %d"))  # "May 24" format
        return dates
    
    def open_channel(self):
        """Open Chrome directly to the channel"""
        print(f"🚀 Opening channel: {CHANNEL_URL}")
        
        if SELENIUM_AVAILABLE:
            try:
                # Setup Chrome with minimal options + suppress errors
                service = Service(ChromeDriverManager().install())
                options = Options()
                options.add_argument("--start-maximized")
                options.add_argument("--disable-blink-features=AutomationControlled")
                options.add_argument("--disable-dev-shm-usage")
                options.add_argument("--no-sandbox")
                options.add_argument("--disable-gpu")
                options.add_argument("--disable-logging")
                options.add_argument("--log-level=3")
                options.add_experimental_option("detach", True)
                options.add_experimental_option("excludeSwitches", ["enable-logging"])
                options.add_experimental_option('useAutomationExtension', False)
                
                self.driver = webdriver.Chrome(service=service, options=options)
                self.driver.get(CHANNEL_URL)
                print("✅ Chrome opened with Selenium")
                
                # Wait longer for Telegram to load
                print("⏳ Waiting for Telegram to load...")
                time.sleep(15)
                
                # Try to maximize window
                try:
                    self.driver.maximize_window()
                    print("🔳 Window maximized")
                except:
                    pass
                
                return True
                
            except Exception as e:
                print(f"❌ Selenium failed: {e}")
                return self._open_with_webbrowser()
        else:
            return self._open_with_webbrowser()
    
    def _open_with_webbrowser(self):
        """Fallback: open with system browser"""
        try:
            webbrowser.open(CHANNEL_URL)
            print("✅ Opened with system browser")
            time.sleep(5)
            return True
        except Exception as e:
            print(f"❌ Failed to open browser: {e}")
            return False
    
    def setup_capture_region(self):
        """Setup screenshot region for messages with better Chrome detection"""
        screen_w, screen_h = pyautogui.size()
        print(f"🖥️ Screen size: {screen_w}x{screen_h}")
        
        # Try to maximize the Chrome window first
        try:
            if self.driver:
                self.driver.maximize_window()
                time.sleep(2)  # Wait for maximize to complete
                print("🔳 Chrome window maximized")
        except:
            pass
        
        # Try multiple methods to find Chrome window
        window_found = False
        
        # Method 1: Try different Chrome window names
        chrome_names = ["Chrome", "Google Chrome", "Chromium", "chrome"]
        
        for name in chrome_names:
            try:
                result = subprocess.run(
                    ["xdotool", "search", "--name", name],
                    capture_output=True, text=True
                )
                
                if result.returncode == 0 and result.stdout.strip():
                    win_ids = result.stdout.strip().split('\n')
                    
                    for win_id in win_ids:
                        geom_result = subprocess.run(
                            ["xdotool", "getwindowgeometry", "--shell", win_id],
                            capture_output=True, text=True
                        )
                        
                        if geom_result.returncode == 0:
                            # Parse geometry
                            geom = {}
                            for line in geom_result.stdout.split('\n'):
                                if '=' in line:
                                    key, value = line.split('=', 1)
                                    geom[key] = int(value)
                            
                            # Check if this window looks reasonable (not minimized)
                            if geom.get('WIDTH', 0) > 500 and geom.get('HEIGHT', 0) > 400:
                                print(f"🪟 Found {name} window: X={geom.get('X', 0)}, Y={geom.get('Y', 0)}, W={geom.get('WIDTH', 0)}, H={geom.get('HEIGHT', 0)}")
                                
                                # Use the full window for Telegram Web
                                # Telegram Web has sidebar on left (~300px) and chat on right
                                self.left = geom.get('X', 0) + 300  # Skip sidebar
                                self.top = geom.get('Y', 0) + 60   # Skip header
                                self.width = geom.get('WIDTH', 0) - 350  # Main chat area
                                self.height = geom.get('HEIGHT', 0) - 120  # Skip header/footer
                                
                                # Validate region
                                if (self.width > 0 and self.height > 0 and 
                                    self.left >= 0 and self.top >= 0 and
                                    self.left + self.width <= screen_w and
                                    self.top + self.height <= screen_h):
                                    
                                    window_found = True
                                    break
                    
                    if window_found:
                        break
                        
            except Exception as e:
                continue
        
        if not window_found:
            print("⚠️ Could not detect Chrome window, using screen-based approach")
            # Use a large portion of the screen (assuming maximized browser)
            self.left = screen_w // 4  # Skip left quarter (sidebar area)
            self.top = 100  # Skip top margin
            self.width = screen_w * 3 // 5  # Use most of the width
            self.height = screen_h - 200  # Use most of the height
        
        # Final validation and adjustment
        if self.left + self.width > screen_w:
            self.width = screen_w - self.left - 50
        if self.top + self.height > screen_h:
            self.height = screen_h - self.top - 50
        
        print(f"📐 Final capture region: {self.left},{self.top} {self.width}x{self.height}")
        
        # Test the region
        try:
            test_img = pyautogui.screenshot(region=(self.left, self.top, self.width, self.height))
            print("✅ Capture region test successful")
            
            # Save test image for debugging
            test_img.save("test_capture.png")
            print("🖼️ Test capture saved as test_capture.png")
            
            return True
        except Exception as e:
            print(f"❌ Capture region test failed: {e}")
            return False
    
    def capture_screen(self):
        """Take screenshot and OCR it"""
        try:
            img = pyautogui.screenshot(region=(self.left, self.top, self.width, self.height))
            gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
            
            # Enhance image for better OCR
            gray = cv2.convertScaleAbs(gray, alpha=1.2, beta=10)
            
            # OCR with custom config
            config = '--oem 3 --psm 6'
            text = pytesseract.image_to_string(gray, config=config)
            return text.strip()
            
        except Exception as e:
            print(f"❌ Screenshot failed: {e}")
            return ""
    
    def scroll_up(self, pixels=400):
        """Scroll up in the message area"""
        center_x = self.left + self.width // 2
        center_y = self.top + self.height // 2
        pyautogui.moveTo(center_x, center_y, duration=0.1)
        pyautogui.scroll(pixels)
    
    def scroll_down(self, pixels=400):
        """Scroll down in the message area"""
        center_x = self.left + self.width // 2
        center_y = self.top + self.height // 2
        pyautogui.moveTo(center_x, center_y, duration=0.1)
        pyautogui.scroll(-pixels)
    
    def find_messages_for_date(self, date_str, max_steps=100):
        """Find all messages for a specific date with robust scrolling"""
        print(f"🔍 Searching for messages from {date_str}...")
        messages = []
        consecutive_empty = 0
        
        # Start by scrolling to recent messages (scroll down a bit first)
        print("📜 Positioning at recent messages...")
        for _ in range(5):
            self.scroll_down(200)
            time.sleep(0.3)
        
        for step in range(max_steps):
            text = self.capture_screen()
            
            # Debug: show what we're seeing every 10 steps
            if step % 10 == 0:
                preview = text.replace('\n', ' ')[:100] if text else "No text captured"
                print(f"   Step {step}: {preview}...")
            
            if text:
                consecutive_empty = 0
                
                # Check for target date (flexible matching)
                if (date_str in text or 
                    date_str.lower() in text.lower() or
                    any(word in text for word in date_str.split())):
                    
                    print(f"✅ Found {date_str} messages at step {step}")
                    messages.append(text)
                    
                    # Collect more messages around this date
                    print("📥 Collecting additional messages...")
                    for i in range(8):  # Get more context
                        if i < 4:
                            self.scroll_up(200)  # Look for older messages
                        else:
                            self.scroll_down(200)  # Look for newer messages
                        time.sleep(0.4)
                        
                        more_text = self.capture_screen()
                        if more_text and (date_str in more_text or 
                                        any(word in more_text for word in date_str.split())):
                            messages.append(more_text)
                    
                    print(f"📥 Collected {len(messages)} message blocks for {date_str}")
                    return messages
                
                # Check if we see any dates at all to know we're in the right area
                date_patterns = [
                    r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\b',
                    r'\b\d{1,2}:\d{2}\b',  # Time stamps
                    r'\bToday\b', r'\bYesterday\b'
                ]
                
                has_date_info = any(re.search(pattern, text, re.IGNORECASE) for pattern in date_patterns)
                if not has_date_info:
                    consecutive_empty += 1
                    if consecutive_empty > 15:
                        print("⚠️ No date information found for a while, might be in wrong area")
                        # Try scrolling in the opposite direction
                        for _ in range(10):
                            self.scroll_down(400)
                            time.sleep(0.3)
                        consecutive_empty = 0
            else:
                consecutive_empty += 1
                if consecutive_empty > 20:
                    print("⚠️ Too many empty captures, might be outside message area")
                    break
            
            # Scroll up to find older messages
            self.scroll_up(300)
            time.sleep(0.4)
            
            # Every 25 steps, give status update
            if step % 25 == 0 and step > 0:
                print(f"   ... still searching ({step}/{max_steps}) for {date_str}")
        
        print(f"❌ Completed search for {date_str} - found {len(messages)} messages")
        return messages
    
    def parse_messages(self, raw_messages, date_str):
        """Parse OCR text into structured data"""
        parsed = []
        
        for text_block in raw_messages:
            lines = [line.strip() for line in text_block.split('\n') if line.strip()]
            
            current_msg = {
                'date': date_str,
                'timestamp': '',
                'content': '',
                'raw_text': text_block
            }
            
            # Look for time patterns (14:30, 2:30 PM, etc.)
            time_pattern = r'\b\d{1,2}:\d{2}(?:\s*[AP]M)?\b'
            
            for i, line in enumerate(lines):
                if re.search(time_pattern, line):
                    current_msg['timestamp'] = line
                    # Content is usually after timestamp
                    if i + 1 < len(lines):
                        current_msg['content'] = ' '.join(lines[i+1:])
                    break
            
            # If no timestamp found, treat first line as content
            if not current_msg['content'] and lines:
                current_msg['content'] = lines[0]
            
            if current_msg['content']:
                parsed.append(current_msg)
        
        return parsed
    
    def extract_all_messages(self):
        """Extract messages for all target dates with robust fallback"""
        print(f"📥 Extracting messages for last {self.days} days...")
        print(f"🎯 Target dates: {', '.join(self.target_dates)}")
        
        total_found = 0
        
        for i, date_str in enumerate(self.target_dates):
            print(f"\n--- Processing date {i+1}/{len(self.target_dates)}: {date_str} ---")
            
            raw_messages = self.find_messages_for_date(date_str)
            
            if raw_messages:
                parsed = self.parse_messages(raw_messages, date_str)
                self.messages_data.extend(parsed)
                total_found += len(parsed)
                print(f"✅ Added {len(parsed)} messages for {date_str}")
            else:
                print(f"⚠️ No messages found for {date_str}")
                
                # Try alternative date formats
                alt_formats = [
                    date_str.replace(" ", ""),  # "May24"
                    date_str.split()[0],        # Just "May"  
                    date_str.split()[1] if len(date_str.split()) > 1 else "",  # Just "24"
                ]
                
                for alt_date in alt_formats:
                    if alt_date and len(alt_date) > 1:
                        print(f"🔄 Trying alternative format: {alt_date}")
                        alt_messages = self.find_messages_for_date(alt_date, max_steps=50)
                        if alt_messages:
                            parsed = self.parse_messages(alt_messages, date_str)
                            self.messages_data.extend(parsed)
                            total_found += len(parsed)
                            print(f"✅ Found {len(parsed)} messages with alternative format!")
                            break
            
            # Brief pause between dates
            time.sleep(2)
        
        print(f"\n🎯 Extraction complete! Total messages found: {total_found}")
        
        if total_found == 0:
            print("\n🔍 No messages found - trying broader search...")
            # Try a general scroll through recent messages
            self._emergency_message_search()
    
    def _emergency_message_search(self):
        """Emergency fallback - just grab any recent messages we can find"""
        print("🆘 Performing emergency message search...")
        
        # Reset position and scroll through recent messages
        for _ in range(5):
            self.scroll_down(300)
            time.sleep(0.5)
        
        emergency_messages = []
        for step in range(30):  # Limited search
            text = self.capture_screen()
            if text and len(text.strip()) > 10:  # Any substantial text
                emergency_messages.append({
                    'date': 'recent',
                    'timestamp': f'emergency_step_{step}',
                    'content': text[:200],  # First 200 chars
                    'raw_text': text
                })
                print(f"📥 Emergency capture {step}: {text[:50]}...")
            
            self.scroll_up(200)
            time.sleep(0.3)
        
        if emergency_messages:
            self.messages_data.extend(emergency_messages)
            print(f"🆘 Emergency search found {len(emergency_messages)} message captures")
        else:
            print("🆘 Emergency search found nothing - check if Telegram is loaded properly")
    
    def create_dataframe(self):
        """Convert messages to pandas DataFrame"""
        if self.messages_data:
            df = pd.DataFrame(self.messages_data)
            print(f"📊 Created DataFrame with {len(df)} messages")
            return df
        else:
            print("❌ No messages to create DataFrame")
            return pd.DataFrame()
    
    def minimize_browser(self):
        """Minimize the browser window"""
        try:
            if self.driver:
                self.driver.minimize_window()
            else:
                subprocess.run(["wmctrl", "-r", "Chrome", "-b", "add,hidden"], 
                             check=False, capture_output=True)
            print("🔻 Browser minimized")
        except Exception as e:
            print(f"⚠️ Could not minimize browser: {e}")
    
    def run(self):
        """Main execution method"""
        try:
            print("🎯 Starting Telegram message extraction...")
            
            # Step 1: Open the channel
            if not self.open_channel():
                print("❌ Failed to open channel")
                return pd.DataFrame()
            
            print("✅ Channel opened successfully")
            
            # Step 2: Setup capture region
            if not self.setup_capture_region():
                print("❌ Failed to setup capture region")
                return pd.DataFrame()
            
            print("✅ Capture region configured")
            
            # Give user a moment to see Telegram is loaded
            print("⏳ Waiting for Telegram to be fully ready...")
            time.sleep(5)
            
            # Step 3: Extract messages
            self.extract_all_messages()
            
            # Step 4: Create DataFrame
            df = self.create_dataframe()
            
            # Step 5: Save results
            if not df.empty:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"telegram_messages_{timestamp}.csv"
                df.to_csv(filename, index=False)
                print(f"💾 Saved to {filename}")
            
            # Step 6: Minimize browser
            self.minimize_browser()
            
            return df
            
        except Exception as e:
            print(f"❌ Error during extraction: {e}")
            return pd.DataFrame()
        
        finally:
            # Clean up selenium driver if it exists
            if self.driver:
                try:
                    pass  # Keep browser open but minimized
                except:
                    pass

def main():
    """Run the extractor"""
    print("=" * 60)
    print("🔥 TELEGRAM MESSAGE EXTRACTOR")
    print("=" * 60)
    
    # Create and run extractor
    extractor = TelegramExtractor(days=DAYS_TO_EXTRACT)
    df = extractor.run()
    
    # Show results
    if not df.empty:
        print("\n" + "=" * 60)
        print("✅ EXTRACTION COMPLETE!")
        print("=" * 60)
        print(f"📊 Total messages: {len(df)}")
        print(f"📅 Date range: {DAYS_TO_EXTRACT} days")
        print("\n📋 Sample messages:")
        print(df[['date', 'timestamp', 'content']].head().to_string())
    else:
        print("\n❌ No messages extracted")
    
    return df

if __name__ == "__main__":
    df = main()

🔥 TELEGRAM MESSAGE EXTRACTOR
🎯 Starting Telegram message extraction...
🚀 Opening channel: https://web.telegram.org/k/#-1915245777
✅ Opened with system browser


[33789:33815:0524/184624.882564:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
[33789:33818:0524/184624.941363:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
[33789:33818:0524/184624.941430:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
error: XDG_RUNTIME_DIR is invalid or not set in the environment.
[33830:33830:0524/184625.069557:ERROR:components/viz/service/main/viz_main_impl.cc:183] Exiting GPU process due to errors during initialization
[33789:33923:0524/184625.165141:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaco

✅ Channel opened successfully
🖥️ Screen size: 1920x1200
🪟 Found Chrome window: X=0, Y=0, W=1920, H=1054
📐 Final capture region: 300,60 1570x934
✅ Capture region test successful
🖼️ Test capture saved as test_capture.png
✅ Capture region configured
⏳ Waiting for Telegram to be fully ready...
📥 Extracting messages for last 2 days...
🎯 Target dates: May 24, May 23

--- Processing date 1/2: May 24 ---
🔍 Searching for messages from May 24...
📜 Positioning at recent messages...


KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
"""
Simplified Telegram Channel Message Extractor
Goes directly to OptionRecom Premium Plus channel and extracts messages.
"""

import subprocess
import time
import re
import cv2
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import webbrowser
import warnings
import os

# Suppress warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
import pyautogui
import pytesseract

# Optional selenium imports
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    from webdriver_manager.chrome import ChromeDriverManager
    SELENIUM_AVAILABLE = True
except ImportError:
    SELENIUM_AVAILABLE = False

# Configuration
DAYS_TO_EXTRACT = 2
CHANNEL_URL = "https://web.telegram.org/k/#-1915245777"

class TelegramExtractor:
    def __init__(self, days=DAYS_TO_EXTRACT):
        self.days = days
        self.driver = None
        self.messages_data = []
        self.target_dates = self._get_target_dates()
        
    def _get_target_dates(self):
        """Get list of dates to search for"""
        dates = []
        for i in range(self.days):
            date = datetime.now() - timedelta(days=i)
            dates.append(date.strftime("%b %d"))  # "May 24" format
        return dates
    
    def open_channel(self):
        """Open Chrome directly to the channel"""
        print(f"🚀 Opening channel: {CHANNEL_URL}")
        
        if SELENIUM_AVAILABLE:
            try:
                # Setup Chrome with minimal options + suppress errors
                service = Service(ChromeDriverManager().install())
                options = Options()
                options.add_argument("--start-maximized")
                options.add_argument("--disable-blink-features=AutomationControlled")
                options.add_argument("--disable-dev-shm-usage")
                options.add_argument("--no-sandbox")
                options.add_argument("--disable-gpu")
                options.add_argument("--disable-logging")
                options.add_argument("--log-level=3")
                options.add_experimental_option("detach", True)
                options.add_experimental_option("excludeSwitches", ["enable-logging"])
                options.add_experimental_option('useAutomationExtension', False)
                
                self.driver = webdriver.Chrome(service=service, options=options)
                self.driver.get(CHANNEL_URL)
                print("✅ Chrome opened with Selenium")
                
                # Wait longer for Telegram to load
                print("⏳ Waiting for Telegram to load...")
                time.sleep(15)
                
                # Try to maximize window
                try:
                    self.driver.maximize_window()
                    print("🔳 Window maximized")
                except:
                    pass
                
                return True
                
            except Exception as e:
                print(f"❌ Selenium failed: {e}")
                return self._open_with_webbrowser()
        else:
            return self._open_with_webbrowser()
    
    def _open_with_webbrowser(self):
        """Fallback: open with system browser"""
        try:
            webbrowser.open(CHANNEL_URL)
            print("✅ Opened with system browser")
            time.sleep(5)
            return True
        except Exception as e:
            print(f"❌ Failed to open browser: {e}")
            return False
    
    def setup_capture_region(self):
        """Setup screenshot region for messages with better Chrome detection"""
        screen_w, screen_h = pyautogui.size()
        print(f"🖥️ Screen size: {screen_w}x{screen_h}")
        
        # Try to maximize the Chrome window first
        try:
            if self.driver:
                self.driver.maximize_window()
                time.sleep(2)  # Wait for maximize to complete
                print("🔳 Chrome window maximized")
        except:
            pass
        
        # Try multiple methods to find Chrome window
        window_found = False
        
        # Method 1: Try different Chrome window names
        chrome_names = ["Chrome", "Google Chrome", "Chromium", "chrome"]
        
        for name in chrome_names:
            try:
                result = subprocess.run(
                    ["xdotool", "search", "--name", name],
                    capture_output=True, text=True
                )
                
                if result.returncode == 0 and result.stdout.strip():
                    win_ids = result.stdout.strip().split('\n')
                    
                    for win_id in win_ids:
                        geom_result = subprocess.run(
                            ["xdotool", "getwindowgeometry", "--shell", win_id],
                            capture_output=True, text=True
                        )
                        
                        if geom_result.returncode == 0:
                            # Parse geometry
                            geom = {}
                            for line in geom_result.stdout.split('\n'):
                                if '=' in line:
                                    key, value = line.split('=', 1)
                                    geom[key] = int(value)
                            
                            # Check if this window looks reasonable (not minimized)
                            if geom.get('WIDTH', 0) > 500 and geom.get('HEIGHT', 0) > 400:
                                print(f"🪟 Found {name} window: X={geom.get('X', 0)}, Y={geom.get('Y', 0)}, W={geom.get('WIDTH', 0)}, H={geom.get('HEIGHT', 0)}")
                                
                                # Use the full window for Telegram Web
                                # Telegram Web has sidebar on left (~300px) and chat on right
                                self.left = geom.get('X', 0) + 300  # Skip sidebar
                                self.top = geom.get('Y', 0) + 60   # Skip header
                                self.width = geom.get('WIDTH', 0) - 350  # Main chat area
                                self.height = geom.get('HEIGHT', 0) - 120  # Skip header/footer
                                
                                # Validate region
                                if (self.width > 0 and self.height > 0 and 
                                    self.left >= 0 and self.top >= 0 and
                                    self.left + self.width <= screen_w and
                                    self.top + self.height <= screen_h):
                                    
                                    window_found = True
                                    break
                    
                    if window_found:
                        break
                        
            except Exception as e:
                continue
        
        if not window_found:
            print("⚠️ Could not detect Chrome window, using screen-based approach")
            # Use a large portion of the screen (assuming maximized browser)
            self.left = screen_w // 4  # Skip left quarter (sidebar area)
            self.top = 100  # Skip top margin
            self.width = screen_w * 3 // 5  # Use most of the width
            self.height = screen_h - 200  # Use most of the height
        
        # Final validation and adjustment
        if self.left + self.width > screen_w:
            self.width = screen_w - self.left - 50
        if self.top + self.height > screen_h:
            self.height = screen_h - self.top - 50
        
        print(f"📐 Final capture region: {self.left},{self.top} {self.width}x{self.height}")
        
        # Test the region
        try:
            test_img = pyautogui.screenshot(region=(self.left, self.top, self.width, self.height))
            print("✅ Capture region test successful")
            
            # Save test image for debugging
            test_img.save("test_capture.png")
            print("🖼️ Test capture saved as test_capture.png")
            
            return True
        except Exception as e:
            print(f"❌ Capture region test failed: {e}")
            return False
    
    def capture_screen(self):
        """Take screenshot and OCR it"""
        try:
            img = pyautogui.screenshot(region=(self.left, self.top, self.width, self.height))
            gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
            
            # Enhance image for better OCR
            gray = cv2.convertScaleAbs(gray, alpha=1.2, beta=10)
            
            # OCR with custom config
            config = '--oem 3 --psm 6'
            text = pytesseract.image_to_string(gray, config=config)
            return text.strip()
            
        except Exception as e:
            print(f"❌ Screenshot failed: {e}")
            return ""
    
    def scroll_up(self, pixels=400):
        """Scroll up in the message area"""
        center_x = self.left + self.width // 2
        center_y = self.top + self.height // 2
        pyautogui.moveTo(center_x, center_y, duration=0.1)
        pyautogui.scroll(pixels)
    
    def scroll_down(self, pixels=400):
        """Scroll down in the message area"""
        center_x = self.left + self.width // 2
        center_y = self.top + self.height // 2
        pyautogui.moveTo(center_x, center_y, duration=0.1)
        pyautogui.scroll(-pixels)
    
    def find_messages_for_date(self, date_str, max_steps=100):
        """Find all messages for a specific date with robust scrolling"""
        print(f"🔍 Searching for messages from {date_str}...")
        messages = []
        consecutive_empty = 0
        
        # Start by scrolling to recent messages (scroll down a bit first)
        print("📜 Positioning at recent messages...")
        for _ in range(5):
            self.scroll_down(200)
            time.sleep(0.3)
        
        for step in range(max_steps):
            text = self.capture_screen()
            
            # Debug: show what we're seeing every 10 steps
            if step % 10 == 0:
                preview = text.replace('\n', ' ')[:100] if text else "No text captured"
                print(f"   Step {step}: {preview}...")
            
            if text:
                consecutive_empty = 0
                
                # Check for target date (flexible matching)
                if (date_str in text or 
                    date_str.lower() in text.lower() or
                    any(word in text for word in date_str.split())):
                    
                    print(f"✅ Found {date_str} messages at step {step}")
                    messages.append(text)
                    
                    # Collect more messages around this date
                    print("📥 Collecting additional messages...")
                    for i in range(8):  # Get more context
                        if i < 4:
                            self.scroll_up(200)  # Look for older messages
                        else:
                            self.scroll_down(200)  # Look for newer messages
                        time.sleep(0.4)
                        
                        more_text = self.capture_screen()
                        if more_text and (date_str in more_text or 
                                        any(word in more_text for word in date_str.split())):
                            messages.append(more_text)
                    
                    print(f"📥 Collected {len(messages)} message blocks for {date_str}")
                    return messages
                
                # Check if we see any dates at all to know we're in the right area
                date_patterns = [
                    r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\b',
                    r'\b\d{1,2}:\d{2}\b',  # Time stamps
                    r'\bToday\b', r'\bYesterday\b'
                ]
                
                has_date_info = any(re.search(pattern, text, re.IGNORECASE) for pattern in date_patterns)
                if not has_date_info:
                    consecutive_empty += 1
                    if consecutive_empty > 15:
                        print("⚠️ No date information found for a while, might be in wrong area")
                        # Try scrolling in the opposite direction
                        for _ in range(10):
                            self.scroll_down(400)
                            time.sleep(0.3)
                        consecutive_empty = 0
            else:
                consecutive_empty += 1
                if consecutive_empty > 20:
                    print("⚠️ Too many empty captures, might be outside message area")
                    break
            
            # Scroll up to find older messages
            self.scroll_up(300)
            time.sleep(0.4)
            
            # Every 25 steps, give status update
            if step % 25 == 0 and step > 0:
                print(f"   ... still searching ({step}/{max_steps}) for {date_str}")
        
        print(f"❌ Completed search for {date_str} - found {len(messages)} messages")
        return messages
    
    def parse_messages(self, raw_messages, date_str):
        """Parse OCR text into structured data"""
        parsed = []
        
        for text_block in raw_messages:
            lines = [line.strip() for line in text_block.split('\n') if line.strip()]
            
            current_msg = {
                'date': date_str,
                'timestamp': '',
                'content': '',
                'raw_text': text_block
            }
            
            # Look for time patterns (14:30, 2:30 PM, etc.)
            time_pattern = r'\b\d{1,2}:\d{2}(?:\s*[AP]M)?\b'
            
            for i, line in enumerate(lines):
                if re.search(time_pattern, line):
                    current_msg['timestamp'] = line
                    # Content is usually after timestamp
                    if i + 1 < len(lines):
                        current_msg['content'] = ' '.join(lines[i+1:])
                    break
            
            # If no timestamp found, treat first line as content
            if not current_msg['content'] and lines:
                current_msg['content'] = lines[0]
            
            if current_msg['content']:
                parsed.append(current_msg)
        
        return parsed
    
    def extract_all_messages(self):
        """Extract messages for all target dates with robust fallback"""
        print(f"📥 Extracting messages for last {self.days} days...")
        print(f"🎯 Target dates: {', '.join(self.target_dates)}")
        
        total_found = 0
        
        for i, date_str in enumerate(self.target_dates):
            print(f"\n--- Processing date {i+1}/{len(self.target_dates)}: {date_str} ---")
            
            raw_messages = self.find_messages_for_date(date_str)
            
            if raw_messages:
                parsed = self.parse_messages(raw_messages, date_str)
                self.messages_data.extend(parsed)
                total_found += len(parsed)
                print(f"✅ Added {len(parsed)} messages for {date_str}")
            else:
                print(f"⚠️ No messages found for {date_str}")
                
                # Try alternative date formats
                alt_formats = [
                    date_str.replace(" ", ""),  # "May24"
                    date_str.split()[0],        # Just "May"  
                    date_str.split()[1] if len(date_str.split()) > 1 else "",  # Just "24"
                ]
                
                for alt_date in alt_formats:
                    if alt_date and len(alt_date) > 1:
                        print(f"🔄 Trying alternative format: {alt_date}")
                        alt_messages = self.find_messages_for_date(alt_date, max_steps=50)
                        if alt_messages:
                            parsed = self.parse_messages(alt_messages, date_str)
                            self.messages_data.extend(parsed)
                            total_found += len(parsed)
                            print(f"✅ Found {len(parsed)} messages with alternative format!")
                            break
            
            # Brief pause between dates
            time.sleep(2)
        
        print(f"\n🎯 Extraction complete! Total messages found: {total_found}")
        
        if total_found == 0:
            print("\n🔍 No messages found - trying broader search...")
            # Try a general scroll through recent messages
            self._emergency_message_search()
    
    def _emergency_message_search(self):
        """Emergency fallback - just grab any recent messages we can find"""
        print("🆘 Performing emergency message search...")
        
        # Reset position and scroll through recent messages
        for _ in range(5):
            self.scroll_down(300)
            time.sleep(0.5)
        
        emergency_messages = []
        for step in range(30):  # Limited search
            text = self.capture_screen()
            if text and len(text.strip()) > 10:  # Any substantial text
                emergency_messages.append({
                    'date': 'recent',
                    'timestamp': f'emergency_step_{step}',
                    'content': text[:200],  # First 200 chars
                    'raw_text': text
                })
                print(f"📥 Emergency capture {step}: {text[:50]}...")
            
            self.scroll_up(200)
            time.sleep(0.3)
        
        if emergency_messages:
            self.messages_data.extend(emergency_messages)
            print(f"🆘 Emergency search found {len(emergency_messages)} message captures")
        else:
            print("🆘 Emergency search found nothing - check if Telegram is loaded properly")
    
    def create_dataframe(self):
        """Convert messages to pandas DataFrame"""
        if self.messages_data:
            df = pd.DataFrame(self.messages_data)
            print(f"📊 Created DataFrame with {len(df)} messages")
            return df
        else:
            print("❌ No messages to create DataFrame")
            return pd.DataFrame()
    
    def minimize_browser(self):
        """Minimize the browser window"""
        try:
            if self.driver:
                self.driver.minimize_window()
            else:
                subprocess.run(["wmctrl", "-r", "Chrome", "-b", "add,hidden"], 
                             check=False, capture_output=True)
            print("🔻 Browser minimized")
        except Exception as e:
            print(f"⚠️ Could not minimize browser: {e}")
    
    def run(self):
        """Main execution method"""
        try:
            print("🎯 Starting Telegram message extraction...")
            
            # Step 1: Open the channel
            if not self.open_channel():
                print("❌ Failed to open channel")
                return pd.DataFrame()
            
            print("✅ Channel opened successfully")
            
            # Step 2: Setup capture region
            if not self.setup_capture_region():
                print("❌ Failed to setup capture region")
                return pd.DataFrame()
            
            print("✅ Capture region configured")
            
            # Give user a moment to see Telegram is loaded
            print("⏳ Waiting for Telegram to be fully ready...")
            time.sleep(5)
            
            # Step 3: Extract messages
            self.extract_all_messages()
            
            # Step 4: Create DataFrame
            df = self.create_dataframe()
            
            # Step 5: Save results
            if not df.empty:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"telegram_messages_{timestamp}.csv"
                df.to_csv(filename, index=False)
                print(f"💾 Saved to {filename}")
            
            # Step 6: Minimize browser
            self.minimize_browser()
            
            return df
            
        except Exception as e:
            print(f"❌ Error during extraction: {e}")
            return pd.DataFrame()
        
        finally:
            # Clean up selenium driver if it exists
            if self.driver:
                try:
                    pass  # Keep browser open but minimized
                except:
                    pass

def main():
    """Run the extractor"""
    print("=" * 60)
    print("🔥 TELEGRAM MESSAGE EXTRACTOR")
    print("=" * 60)
    
    # Create and run extractor
    extractor = TelegramExtractor(days=DAYS_TO_EXTRACT)
    df = extractor.run()
    
    # Show results
    if not df.empty:
        print("\n" + "=" * 60)
        print("✅ EXTRACTION COMPLETE!")
        print("=" * 60)
        print(f"📊 Total messages: {len(df)}")
        print(f"📅 Date range: {DAYS_TO_EXTRACT} days")
        print("\n📋 Sample messages:")
        print(df[['date', 'timestamp', 'content']].head().to_string())
    else:
        print("\n❌ No messages extracted")
    
    return df

if __name__ == "__main__":
    df = main()

🔥 TELEGRAM MESSAGE EXTRACTOR
🎯 Starting Telegram message extraction...
🚀 Opening channel: https://web.telegram.org/k/#-1915245777
✅ Opened with system browser


[9645:9671:0524/122607.276509:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
[9645:9674:0524/122607.335739:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
[9645:9674:0524/122607.335795:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/telegram-ocr/var/run/dbus/system_bus_socket: No such file or directory
error: XDG_RUNTIME_DIR is invalid or not set in the environment.
[9685:9685:0524/122607.458160:ERROR:components/viz/service/main/viz_main_impl.cc:183] Exiting GPU process due to errors during initialization
[9645:9782:0524/122607.570354:ERROR:dbus/bus.cc:408] Failed to connect to the bus: Failed to connect to socket /home/chris_s_dodd/anaconda3/envs/

✅ Channel opened successfully
🖥️ Screen size: 1920x1200
🪟 Window geometry: X=10, Y=10, W=10, H=10
⚠️ Invalid dimensions: 360,110 -390x-190
📐 Using safe fallback region: 480,120 800x600
✅ Capture region test successful
✅ Capture region configured
⏳ Waiting for Telegram to be fully ready...
📥 Extracting messages for last 2 days...
🔍 Searching for messages from May 24...


KeyboardInterrupt: 